In [13]:
%run ../../Setup.ipynb
import ast
import json
from bdcc.database.connection import database_connector as connector

In [14]:
def to_dict(data) -> dict:
    data_dict = ast.literal_eval(data) #convert dict string to dict
    return data_dict

In [15]:
def write_match(match):
    """
    Function to create entries in the database.
    params: *match: dict of match data
    """
    match_dict = to_dict(match)
    db_con = connector.DatabaseConnector()
    db_con.connect()
    db_con.create(match_dict) # create new entry in database
    db_con.disconnect()

In [16]:
def delete_match(match_id):
    """
    Function to delete entries in the database.
    params: *match_id: id of the match to be deleted
    """
    db_con = connector.DatabaseConnector()
    db_con.connect()
    db_con.remove(match_id)
    db_con.disconnect()

In [17]:
def train_model():
    pass

In [18]:
def predict():
    pass